In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn.preprocessing import LabelEncoder

# Defina o caminho do arquivo CSV
caminho_arquivo = r'C:\Users\Miguel\PycharmProjects\lpiproject2024\DataSet.csv'

# Ler o arquivo CSV, pulando a primeira linha, que é o cabeçalho da tabela
df = pd.read_csv(caminho_arquivo, delimiter=';', header=1)

# Acessar a coluna "DescricaoProb" diretamente pelo nome
strings_descricao_prob = df['DescricaoProb'].values
strings_como_reproduzir = df['ComoReproduzir'].values

# strings_descricao_prob é agora um array NumPy contendo os valores da coluna "DescricaoProb"
#print(strings_descricao_prob)
#print(strings_como_reproduzir)

# Embedding model
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')
embeddings_descricao_prob = model.encode(strings_descricao_prob)
#embeddings_como_reproduzir = model.encode(strings_como_reproduzir)


# Clustering
kmeans = KMeans(n_clusters=4, random_state=42)
df['Classificacao'] = kmeans.fit_predict(embeddings_descricao_prob)  # Adiciona diretamente ao DataFrame

# Definição do dicionário com os labels dos clusters
cluster_labels = {0: 'Cluster A', 1: 'Cluster B', 2: 'Cluster C', 3: 'Cluster D'}

# Mapeia os rótulos dos clusters para descrições mais significativas e substitui a coluna 'Classificacao'
df['Departamento'] = df['Classificacao'].map(cluster_labels)

# Remover as colunas indesejadas
colunas_para_remover = ['Unnamed: 9', 'Unnamed: 10']
df.drop(colunas_para_remover, axis=1, inplace=True)

# Salva o DataFrame atualizado em um arquivo CSV
df.to_csv('updated_dataset.csv', sep=';', index=False, encoding='utf-8')

# t-SNE para visualização, igual ao que já estava no seu código
tsne = TSNE(n_components=2, random_state=42)
tsne_embeddings = tsne.fit_transform(embeddings_descricao_prob)
plt.figure(figsize=(10, 8))
scatter = plt.scatter(tsne_embeddings[:, 0], tsne_embeddings[:, 1], c=df['Classificacao'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Classificacao')
plt.title('t-SNE Visualization of Clusters')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()

# Preparação dos dados para classificação
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Classificacao'])
X = embeddings_descricao_prob

# Divisão dos dados para treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Classificador
classifier = RandomForestClassifier(max_depth=10, random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
fig = px.imshow(
    cm,
    x=[f'Cluster {cls}' for cls in label_encoder.classes_],
    y=[f'Cluster {cls}' for cls in label_encoder.classes_],
    text_auto=True,
    labels=dict(x="Predicted Label", y="True Label"),
    color_continuous_scale='Blues',
    title="Confusion Matrix"
)
fig.show()